In [25]:
import tqdm
from call_model import call_chat_once
import asyncio
import httpx
from uuid import uuid4
import json
import datasets
from typing import Optional

In [26]:
eval_dataset = datasets.load_dataset("m-ric/huggingface_doc_qa_eval", split="train") # or load from data/test.csv

Generating train split: 100%|██████████| 65/65 [00:00<00:00, 4694.52 examples/s]


In [ ]:
def run_rag_tests(
    eval_dataset: datasets.Dataset,
    llm,
    output_file: str,
    verbose: Optional[bool] = True,
    test_settings: Optional[str] = None,  # To document the test settings used
):
    """Runs RAG tests on the given dataset and saves the results to the given output file."""
    try:  # load previous generations if they exist
        with open(output_file, "r") as f:
            outputs = json.load(f)
    except:
        outputs = []

    for example in tqdm(eval_dataset):
        question = example["question"]
        if question in [output["question"] for output in outputs]:
            continue
        
        payload = {
            "question": question,
            "session_id": str(uuid4().hex()),
            "chat_history": [
            ]
        }
        # answer, relevant_docs = answer_with_rag(question, llm, knowledge_index, reranker=reranker)
        answer = asyncio.run(call_chat_once(payload))
        if verbose:
            print("=======================================================")
            print(f"Question: {question}")
            print(f"Answer: {answer}")
            print(f'True answer: {example["answer"]}')
        result = {
            "question": question,
            "true_answer": example["answer"],
            "source_doc": example["source_doc"],
            "generated_answer": answer
        }
        if test_settings:
            result["test_settings"] = test_settings
        outputs.append(result)

        with open(output_file, "w") as f:
            json.dump(outputs, f)